In [1]:
import keras_nlp

2023-06-18 15:45:56.954908: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import tensorflow as tf

In [3]:
import pandas as pd
import numpy as np

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [5]:
!ls

Classification_Metrics.md             Untitled.ipynb
CountVectorizer.ipynb                 nlp-getting-started
Disaster_Tweets_with_TensorFlow.ipynb train_test_split.ipynb
NLP.md


In [6]:
train_csv_df = pd.read_csv("nlp-getting-started/train.csv")

In [7]:
train_df, validation_df = train_test_split(train_csv_df, train_size=0.85, random_state=1)

In [8]:
train_df[:10]

,id,keyword,location,text,target
6406,9157,suicide%20bomber,NaN,News: 'Islamic State claims suicide bombing at...,1
2774,3990,devastation,"Denver, CO",$10M SETTLEMENT attained using our illustratio...,1
5050,7198,natural%20disaster,Los Angeles,On the sneak America has us spoiled. A natural...,1
1851,2661,crush,"Cleveland, Ohio",My woman crush wedneday goes to the beautiful ...,0
2320,3336,demolished,"Dublin, Ireland",Home2 Suites offices are coming to Salvi's Bis...,0
4192,5955,hazard,Chicago,CONFIRMED: Sanchez Hazard and Bolasie will be ...,1
5190,7410,obliterated,Tennessee,WACKOES like #MicheleBachman predict the WORLD...,0
6948,9967,tsunami,NaN,she keep it wet like tsunami.,0
623,896,bioterrorism,"Philadelphia, PA",@MeyerBjoern @thelonevirologi @MackayIM of a m...,1
5211,7443,obliterated,don't buy the s*n,Just absolutely obliterated a moth my new purc...,0


In [9]:
text_vectorizer, keyword_vectorizer, location_vectorizer = [CountVectorizer()] * 3

In [10]:
train_text_word_count_vectors = text_vectorizer.fit_transform(train_df["text"].fillna(""))
val_text_word_count_vectors = text_vectorizer.transform(validation_df["text"].fillna(""))
train_text_word_count_vectors.shape, val_text_word_count_vectors.shape

((6471, 19365), (1142, 19365))

In [11]:
train_keyword_word_count_vectors = keyword_vectorizer.fit_transform(train_df["keyword"].fillna(""))
val_keyword_word_count_vectors = keyword_vectorizer.transform(validation_df["keyword"].fillna(""))
train_keyword_word_count_vectors.shape, val_keyword_word_count_vectors.shape

((6471, 239), (1142, 239))

In [12]:
train_location_word_count_vectors = location_vectorizer.fit_transform(train_df["location"].fillna(""))
val_location_word_count_vectors = location_vectorizer.transform(validation_df["location"].fillna(""))
train_location_word_count_vectors.shape, val_location_word_count_vectors.shape

((6471, 2932), (1142, 2932))

In [13]:
train_X = np.hstack((train_keyword_word_count_vectors.todense(), 
                     train_location_word_count_vectors.todense(),
                     train_text_word_count_vectors.todense()))
train_X.shape

(6471, 22536)

In [14]:
val_X = np.hstack((val_keyword_word_count_vectors.todense(), 
                     val_location_word_count_vectors.todense(),
                     val_text_word_count_vectors.todense()))
val_X.shape

(1142, 22536)

In [15]:
model_1 = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

In [16]:
model_1.compile(loss=tf.keras.losses.BinaryCrossentropy(), 
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.BinaryAccuracy(),
                         tf.keras.metrics.BinaryCrossentropy(),
                         tf.keras.metrics.AUC()])

In [17]:
model_1.fit(x=train_X, y=train_df["target"], epochs=50, 
            validation_data=(val_X, validation_df["target"]))

Epoch 1/50
203/203 [==============================] - 2s 5ms/step - loss: 0.6429 - binary_accuracy: 0.7255 - binary_crossentropy: 0.6429 - auc: 0.7707 - val_loss: 0.6008 - val_binary_accuracy: 0.7522 - val_binary_crossentropy: 0.6008 - val_auc: 0.8210
Epoch 2/50
203/203 [==============================] - 1s 5ms/step - loss: 0.5497 - binary_accuracy: 0.8212 - binary_crossentropy: 0.5497 - auc: 0.8751 - val_loss: 0.5516 - val_binary_accuracy: 0.7688 - val_binary_crossentropy: 0.5516 - val_auc: 0.8372
Epoch 3/50
203/203 [==============================] - 1s 5ms/step - loss: 0.4885 - binary_accuracy: 0.8509 - binary_crossentropy: 0.4885 - auc: 0.9056 - val_loss: 0.5207 - val_binary_accuracy: 0.7820 - val_binary_crossentropy: 0.5207 - val_auc: 0.8479
Epoch 4/50
203/203 [==============================] - 1s 5ms/step - loss: 0.4436 - binary_accuracy: 0.8690 - binary_crossentropy: 0.4436 - auc: 0.9245 - val_loss: 0.4999 - val_binary_accuracy: 0.7933 - val_binary_crossentropy: 0.4999 - val_auc:

203/203 [==============================] - 1s 4ms/step - loss: 0.1197 - binary_accuracy: 0.9770 - binary_crossentropy: 0.1197 - auc: 0.9983 - val_loss: 0.4445 - val_binary_accuracy: 0.8004 - val_binary_crossentropy: 0.4445 - val_auc: 0.8634
Epoch 34/50
203/203 [==============================] - 1s 5ms/step - loss: 0.1158 - binary_accuracy: 0.9785 - binary_crossentropy: 0.1158 - auc: 0.9985 - val_loss: 0.4456 - val_binary_accuracy: 0.8004 - val_binary_crossentropy: 0.4456 - val_auc: 0.8631
Epoch 35/50
203/203 [==============================] - 1s 6ms/step - loss: 0.1121 - binary_accuracy: 0.9791 - binary_crossentropy: 0.1121 - auc: 0.9986 - val_loss: 0.4474 - val_binary_accuracy: 0.7986 - val_binary_crossentropy: 0.4474 - val_auc: 0.8627
Epoch 36/50
203/203 [==============================] - 1s 6ms/step - loss: 0.1084 - binary_accuracy: 0.9805 - binary_crossentropy: 0.1084 - auc: 0.9988 - val_loss: 0.4486 - val_binary_accuracy: 0.7986 - val_binary_crossentropy: 0.4486 - val_auc: 0.8626


In [18]:
train_X

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

17602216/17602216 [==============================] - 0s 0us/step


In [27]:
text_tokenizer = keras_nlp.models.BertTokenizer.from_preset("bert_tiny_en_uncased")

In [36]:
text_BERT_preprocessor = keras_nlp.models.BertPreprocessor(text_tokenizer, sequence_length=64)

In [53]:
text_BERT_input = text_BERT_preprocessor(x=train_df["text"])

In [51]:
validation_BERT_input = text_BERT_preprocessor(x=validation_df["text"], y=validation_df["target"])

In [52]:
text_BERT_input

({'token_ids': <tf.Tensor: shape=(6471, 64), dtype=int32, numpy=
  array([[  101,  2739,  1024, ...,     0,     0,     0],
         [  101,  1002,  2184, ...,     0,     0,     0],
         [  101,  2006,  1996, ...,     0,     0,     0],
         ...,
         [  101, 19183,  9451, ...,     0,     0,     0],
         [  101,  2074,  3294, ...,     0,     0,     0],
         [  101,  6168, 15333, ...,     0,     0,     0]], dtype=int32)>,
  'segment_ids': <tf.Tensor: shape=(6471, 64), dtype=int32, numpy=
  array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>,
  'padding_mask': <tf.Tensor: shape=(6471, 64), dtype=bool, numpy=
  array([[ True,  True,  True, ..., False, False, False],
         [ True,  True,  True, ..., False, False, False],
         [ True,  True,  True, ..., False, False, False],
         ...,
 

In [ ]:
tiny_classifier = keras_nlp.models.AlbertClassifier.from_preset(
    "albert_base_en_uncased",
    num_classes=2,
)

tiny_classifier.fit(
    x=train_df["text"],
    y=train_df["target"],
    validation_data=(validation_df["text"], validation_df["target"]),
    epochs=5
)

46778848/46778848 [==============================] - 1s 0us/step


2023-06-18 16:50:24.563014: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'albert_preprocessor/multi_segment_packer_4/Trimmer/Trim/RoundRobinTrimmer/generate_mask/while/Tile/multiples/albert_preprocessor/multi_segment_packer_4/Trimmer/Trim/RoundRobinTrimmer/generate_mask/strided_slice' with dtype int32
	 [[{{node albert_preprocessor/multi_segment_packer_4/Trimmer/Trim/RoundRobinTrimmer/generate_mask/while/Tile/multiples/albert_preprocessor/multi_segment_packer_4/Trimmer/Trim/RoundRobinTrimmer/generate_mask/strided_slice}}]]
2023-06-18 16:50:24.566667: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'albert_preprocessor

Epoch 1/5
